In [2]:
%matplotlib inline 
from interval import interval
import matplotlib.pyplot as plt
from sympy import(
    init_printing,
    var,
    exp,
    cos,
    sin,
    log,
    expand_log
)
from sympy.utilities.lambdify import lambdify 
from sympy.plotting import plot
init_printing()
import numpy as np

from ivmat import ivmat as ip
from fmat import fmat
from krawczyk import Krawczyk
from symbol_derivative import(
    get_f_df_ddf_from_symbol_representation,
    get_global_minima_from_f_expr,
    calc_f_expr_meshgrid,
    plot3D,
    visualize_optimization_log,
)
from nelder_mead import NelderMead

In [24]:
alpha, beta, k = args = var("alpha, beta, k")
t = var("t")
z = var("z")
y_0 = var("y_0")

curve_expr = alpha / (1 + beta * exp(-k*t))
display(curve_expr)
true_curve_expr = curve_expr.subs([(alpha, 1.0), (beta, 0.135), (k, 0.182)])
display(true_curve_expr)


[(0, 0.88105726872246692), (5, 0.94845995547346873), (10, 0.97859473126394758), (15, 0.99127224036313077)]


In [25]:
plot(true_curve_expr, (t, 0, 30))
true_curve = lambdify(t, true_curve_expr)
sample_data = [(time, true_curve(time)) for time in np.arange(0, 20, 5)]
print(sample_data)

In [26]:
loss_expr = 0
for time, value in sample_data:
    pred_value = curve_expr.subs(t, time)
    loss_expr += (value - pred_value) ** 2
f_expr = loss_expr
display(f_expr)

seed_num = 431
np.random.seed(seed_num)
low = 0.01
high = 2.5
points = np.random.uniform(low=low, high=high, size=(len(args)+1, len(args)))
np_func = lambdify([args], f_expr, modules=np) 

In [27]:
nelder = NelderMead(np_func, points)
for i in range(1, 500+1):
    nelder.update_points()
local_minima = nelder.func(nelder.points[0])
print local_minima

1.20794326112e-30


In [28]:
f, df, ddf = get_f_df_ddf_from_symbol_representation(f_expr, args)

In [31]:
X = ip([
    [interval[0.996, 1.0321]],
    [interval[0.124, 0.1384]], 
    [interval[0.11691, 0.207]]
])

# X = ip([
#     [interval[0.5, 1.6]],
#     [interval[0.5, 3.0]], 
#     [interval[0.6, 3.33]]
# ])

f(X)

In [32]:
kra = Krawczyk(df, ddf, X)

In [35]:
import time
st = time.time()
ans_boxes, S_num_list, T_num_list, U_num_list, animation_box = kra.find_global_minimum(
    f, tmp_min_sup=local_minima ,trace=True, cnt_max=1000)
print("{}[sec]".format(time.time() - st))

Loop end. cnt:132, len(S):0, len(T):2, len(U):0

132
---------- 最終的なS[:10] -----------
[]
---------- 最終的なU[:10] -----------
[]
---------- 最終的なT -----------
[[[interval([0.9999455008955325, 1.0000545125000473])],
  [interval([0.13494543921196603, 0.1350545736736702])],
  [interval([0.1817774697429932, 0.18222245554662578])]],
 [[interval([0.9999755256106505, 1.000024475068816])],
  [interval([0.13497548836518056, 0.13502451229181328])],
  [interval([0.18190004557249084, 0.1820999508861647])]]]
11.042495966[sec]


In [10]:
if len(args) == 2: 
    visualize_optimization_log(kra, f, animation_box, skip=200)